- Fragmentation (from [here](https://dl.acm.org/citation.cfm?id=2893731))
- $\text{fragmentation}(x) = 1 - \frac{x \times\#(\text{free chuncks of size } x)}{\text{free space}}$

In [1]:
%matplotlib ipympl
import random

https://github.com/matplotlib/jupyter-matplotlib

In [2]:
def add_var(alloc_size=30, alloc_time=1, dealloc_time=10):
    return {"alloc_size":alloc_size, "alloc_time":alloc_time, "dealloc_time":dealloc_time}

In [49]:
class memory:
    def __init__(self, size):
        self.size = size # memory size in bytes
        self.free_mem = size
        self.memory = [ False ] * size
    
    def __str__(self):
        str1 = "size: {}\n".format(str(self.size))
        str1 += str(list(int(i) for i in self.memory))
        return str1
                
    def writeAllocation(self, position, size):
        self.memory[position: position+size] = [True] * size
            
    def isThereSpace(self, position, size):
        if position+size > self.size:
            return 0
        return sum(self.memory[position: position+size]) == 0

In [50]:
class random_allocated_memory(memory):
    def allocate(self, size):
        allocated = False
        while not allocated:
            pick_place_to_allocate = random.randint(0,self.size-1-size)
            if self.isThereSpace(pick_place_to_allocate, size):
                self.writeAllocation(pick_place_to_allocate, size)
                self.free_mem -= size
                allocated = True

In [94]:
class mpattern:
    'Allocation pattern'
    
    def __init__(self, size, memory_class=random_allocated_memory):
        self.allocation_list = [] # allocation_list is a list of byte sizes of objects to be allocated.
        self.memory = memory_class(size)
    
    def __str__(self):
        str1 = "" #str(memory)
        for var in self.allocation_list:
            #str1 += str(var) + "\n"
            str1 += "alloc_size: " + str(var["alloc_size"]) + "\n"
        return str1
        
    def add_var(self, alloc_size=30, alloc_time=0, dealloc_time=1):
        self.allocation_list.append({"alloc_size":alloc_size, "alloc_time":alloc_time, "dealloc_time":dealloc_time})
    
    def create_random(self, size, number):
        assert(False)
        
    def get_number_free_chunks(self, size):
        num = 0
        i = 0
        while i < self.memory.size:
            if self.memory.isThereSpace(i,size):
                num += 1
                i += size
            else:
                i += 1
        return num
        
    def get_fragmentation(self, size_x):
        if size_x > self.memory.free_mem:
            return 1
        num_free_chunks_of_size_x = self.get_number_free_chunks(size_x)
        return 1-size_x*num_free_chunks_of_size_x/self.memory.free_mem

In [95]:
mem = mpattern(200)

In [96]:
#mem.add_var(1)

In [97]:
mem.memory.allocate(3)

In [98]:
print(mem)

In [99]:
mem.get_fragmentation(30)

0.08629441624365486

In [92]:
mem.memory

In [100]:
memory(30)
random_allocated_memory(3)
print(random_allocated_memory(3))

size: 3
[0, 0, 0]


Some examples:

In [27]:
import matplotlib.pyplot as plt
plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt


plt.rcdefaults()
fig, ax = plt.subplots()

# Example data
people = ('Tom', 'Dick', 'Harry', 'Slim', 'Jim')
y_pos = np.arange(len(people))
performance = 3 + 10 * np.random.rand(len(people))
error = np.random.rand(len(people))

ax.barh(y_pos, performance, xerr=error, align='center',
        color='green', ecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(people)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Performance')
ax.set_title('How fast do you want to go today?')

plt.show()

FigureCanvasNbAgg()

In [28]:
import matplotlib.pyplot as plt
plt.figure()
x = [1,2,3]
y = [4,5,6]
plt.plot(x,y)
plt.show()

FigureCanvasNbAgg()